In [1]:
from contrastive_loss import ContrastiveLoss
from resnet_siamese import ResNet34
from make_dataset import SignaturePairsDataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
import torch
from torch.optim import Adam
import torch.nn.functional as F
from tqdm.notebook import tqdm
import wandb
from torch.optim.lr_scheduler import StepLR
import numpy as np
from PIL import ImageOps

In [2]:
run = wandb.init(
    # Set the project where this run will be logged
    project="signet-reimplement",
    )

wandb: Currently logged in as: dieplstks. Use `wandb login --relogin` to force relogin


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=Image.BILINEAR),
    ImageOps.invert,
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: 1.0 - x),  # Invert the colors
    # transforms.Normalize((0,), (.078,)),  # Calculated
])

x_train_range = range(1, 51)
x_val_range = range(51, 56)
train_dataset = SignaturePairsDataset(
    originals_dir='signatures/signatures/full_org',
    forgeries_dir='signatures/signatures/full_forg',
    x_train_range=x_train_range,
    x_val_range=[],  # Empty range for validation data
    transform=transform
)

val_dataset = SignaturePairsDataset(
    originals_dir='signatures/signatures/full_org',
    forgeries_dir='signatures/signatures/full_forg',
    x_train_range=[],  # Empty range for training data
    x_val_range=x_val_range,
    transform=transform
)

# Create the DataLoaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)


In [5]:
net = ResNet34(in_channels=1, emb_size=128).to(device)
loss = ContrastiveLoss()
optimizer = Adam(net.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=4, gamma=0.5)

In [6]:
epochs = 100

for e in range(epochs):
    train_loader_with_tqdm = tqdm(train_loader, desc=f"Epoch {e+1}/{epochs} (Training)")
    for img1, img2, ys in train_loader_with_tqdm:
        img1 = img1.to(device)
        img2 = img2.to(device)
        ys = ys.to(device)
        o1, o2 = net(img1), net(img2)
        
        optimizer.zero_grad()
        l = loss(o1, o2, ys)
        l.backward()
        optimizer.step()
        
        wandb.log({"batch_loss": l.item()})

    
    with torch.no_grad():
        val_loader_with_tqdm = tqdm(val_loader, desc=f"Epoch {e+1}/{epochs} (Validation)")
        distances = []
        labels = []
    
        for img1, img2, ys in val_loader_with_tqdm:
            img1 = img1.to(device)
            img2 = img2.to(device)
            ys = ys.to(device)
            o1, o2 = net(img1), net(img2)
            distance_batch = F.pairwise_distance(o1, o2).cpu().numpy()
            distances.append(distance_batch)
            labels.append(ys.cpu().numpy())
    
        distances = np.concatenate(distances)
        labels = np.concatenate(labels)
    
        # Scan for the best threshold
        thresholds = np.linspace(start=0, stop=1, num=100)  # Example range, adjust as needed
        best_accuracy = 0
        best_threshold = 0

    for t in thresholds:
        predictions = (distances < t).astype(int)
        accuracy = (predictions == labels).mean()
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = t

    if accuracy < 0.51:
        break
    wandb.log({"best_validation_accuracy": best_accuracy, "best_threshold": best_threshold})
    print(f'Epoch {e+1}/{epochs}, Best Validation Accuracy: {best_accuracy:.4f} at Threshold: {best_threshold}')

    

Epoch 1/100 (Training):   0%|          | 0/216 [00:00<?, ?it/s]

Epoch 1/100 (Validation):   0%|          | 0/22 [00:00<?, ?it/s]

Epoch 1/100, Best Validation Accuracy: 0.7522 at Threshold: 0.05050505050505051


Epoch 2/100 (Training):   0%|          | 0/216 [00:00<?, ?it/s]

Epoch 2/100 (Validation):   0%|          | 0/22 [00:00<?, ?it/s]

...

Epoch 36/100 (Training):   0%|          | 0/216 [00:00<?, ?it/s]

Epoch 36/100 (Validation):   0%|          | 0/22 [00:00<?, ?it/s]

Epoch 36/100, Best Validation Accuracy: 0.6123 at Threshold: 0.31313131313131315


Epoch 37/100 (Training):   0%|          | 0/216 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
wandb.finish()

batch_loss,▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁
best_threshold,▁▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁█▁▁▁▁▃▅▄▅
best_validation_accuracy,▄▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████▁▁▂▂▂▂▂▂▂
batch_loss,0.00068
best_threshold,0.31313
best_validation_accuracy,0.61232


In [8]:
print(accuracy)

0.5224637681159421
